<a href="https://colab.research.google.com/github/nobu-n2002/memorandum-FORTRAN/blob/main/FORTRAN_link_compile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FORTRANのリンク・コンパイル

## 全体像

想定コンパイラ: GNUコンパイラ(gfortran)  
想定環境: Linux系OS

FORTRANファイルが大規模になると、コード修正やコンパイルにて面倒毎が増える。そこで、コンパイルとリンクを使い分けた実行ファイルの作成が有効である。
まず、モジュール用ファイルと、メインファイルを準備し、それぞれコンパイルする。このとき、`-c`オプションを用いて実行ファイルではなくオブジェクトファイルのみを生成する。
```
gfortran -c module.f90 -o module.o
gfortran -c main.f90 -o main.o
```
生成されたオブジェクトファイルをまとめてコンパイルし、実行ファイルにする。
```
gfortran module.o main.o -o program
```


## モジュールファイルの作成

モジュールファイルは、以下の書式で作成する。
```
! module.f90
module {module_name}
  implicit none
  contains ! これ大事
  ! ---
  ! subroutineなどを含むコード
  ! subroutineの数に限りはない
  ! ---
end module {module_name}
```
以下のコードブロックを実行すると、`./content`ディレクトリに`my_module.f90`が生成される。（確認できない場合は、ディレクトリの更新を押すか、新しいコードブロックで`!ls`を実行）

In [49]:
%%writefile my_module.f90
module my_module
  implicit none
  contains

  subroutine hello()
    write(*,*) "Hello, this is your module's subroutine."
  end subroutine hello

  subroutine add(a, b)
    ! local variables
    real,intent(in)::a, b
    real::c
    c = a + b
    write(*,*) a, "+", b, "=", c
    return
  end subroutine add
end module my_module

Overwriting my_module.f90


## モジュールファイルのコンパイル

前のブロックで生成した`my_module.f90`をコンパイルし、オブジェクトファイルを生成する。以下のコードブロックを実行すると、`./content`ディレクトリに`my_module.o`が生成される。（確認できない場合は、ディレクトリの更新を押すか、新しいコードブロックで`!ls`を実行）

なお、`-o`コマンドで出力ファイル名を指定しない場合は、`*.f90`のファイル名を引き継ぎ`*.o`が生成される。

In [50]:
%%bash
gfortran -c my_module.f90 -o my_module.o

## メインファイルの作成

メインファイルは、以下の書式で作成する。
```
! main.f90
program {main_program_name}
  use {module_name} ! これ大事: 先ほど作成したオブジェクトファイル名とする
  implicit none
  ! ---
  ! 以下メインプログラム
  ! module内で定義したsubtoutineがcall可能
  ! call {subroutine}
  ! ---
end module {main_program_name}
```
以下のコードブロックを実行すると、`./content`ディレクトリに`main.f90`が生成される。（確認できない場合は、ディレクトリの更新を押すか、新しいコードブロックで`!ls`を実行）

In [51]:
%%writefile main_program.f90
program main_program
  use my_module
  implicit none
  real::a,b
  a = 2
  b = 4
  call hello()
  call add(a,b)
end program main_program

Overwriting main_program.f90


## メインファイルのコンパイル

前のブロックで生成した`main.f90`をコンパイルし、オブジェクトファイルを生成する。以下のコードブロックを実行すると、`./content`ディレクトリに`main.o`が生成される。（確認できない場合は、ディレクトリの更新を押すか、新しいコードブロックで`!ls`を実行）

In [52]:
%%bash
gfortran -c main_program.f90 -o main_program.o

## モジュールとメインのリンク

以上により生成されたオブジェクトファイルを以下の書式でリンクコンパイルし、実行ファイル`program`を生成する。ただし、オブジェクトファイルの数に限りはない。先ほどのオブジェクトファイルのみの生成時には、`-c`というオプションをつけていたが、今回は実行ファイルを生成するため必要ない。

また、`-o`の後の引数は実行ファイル名を表すため、なんでもよい。ここでは、`program`としている。これを指定しない場合、Linux系では`a.out`、Windows系では`a.exe`が実行ファイルとして生成される。
```
gfortran module1.o module2.o module3.o main.o -o program
```

In [67]:
%%bash
gfortran my_module.o main_program.o -o program

## プログラムの実行と確認

プログラムの実行は、`./{実行ファイル名}`で行う。先ほどのコンパイルでは、実行ファイル名を`program`としたため、
```
./program
```
で実行が可能である。

In [70]:
%%bash
./program

 Hello, this is your module's subroutine.
   2.00000000     +   4.00000000     =   6.00000000    


# Google Colab のマジックコマンド



1.   ファイルの生成  
`%%`をつけた操作がいくつか用意されている。以下はファイルを生成するコマンド。その他のコマンドは、コードブロックで`%magic`を実行すると確認することができる。
```
%%writefile {ファイル名}
ファイルの中身
```

2.   ターミナル操作  
ターミナル操作は、コードブロックで行うことができる。いくつかやり方が存在するが、可読性の観点から、マジックコマンド`%%bash`をコードブロックの1行目に記述する方法がおススメ。`%%sh`を使いシェルスクリプトを組むことも可能。
```
%%bash
{コマンド}
```
```
%%sh
{スクリプト}
```
